<a href="https://colab.research.google.com/github/Phubordin/portfolio-project/blob/main/portfolio-project/P10-Object-Oriented-ATM-System-Financial-Transactions-with-Python-OOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# คำแนะนำ :

My Portfolio Website [Mini Project 11](https://phubordin.github.io/My-Portfolio-Website/project_mini_api_py.html)

---

คุณสามารถลองดึงข้อมูลด้วย API ของเว็บไซต์ (Air Quality API)[https://open-meteo.com/en/docs/air-quality-api] ด้วยโค้ดด้านล่างได้เลยครับ แต่ก่อนอื่นแนะนำให้กด..

`File → Save a copy in Drive` เมื่อ Copy สำเร็จสามารถ

กด `Run all` ที่ซ้ายบนได้เลยครับ (`ต้องรันใน notebook ที่ copy ไปนะครับไม่งั้นจะรันไมไ่ด้`)

---

จากนั้นเมื่อคุณรันเสร็จแล้วคุณจะได้ข้อมูล ฝุ่นในอากาศ (pm10, p,2.5) ไมโครกรัมต่อลูกบาศก์เมตร

รายชั่วโมงย้อนหลัง 90 วันของกรุงเทพ ประเทศไทย -> เป็นไฟล์ Excel

คุณสามารถดาวน์โหลดที่แถบด้านซ้ายรูป File อยู่อันท้ายสุด

เมื่อคลิกเข้าไปคุณจะเห็นชื่อไฟล์ `air_quality.xlsx` สามารถคลิกขวาดาวน์โหลดได้เลย

ลองเล่นดูนะครับ ⭐️

# 🧭 Table of Contents

- **Clear All Variables**
- **Import Library**
- **Configure Coordinates and Files**
- **Retrieve Data for the Past 90 Days**


# Clear All Variables

In [ ]:
# ลบตัวแปรทั้งหมดจาก environment, Runtime > Restart runtime, Cmd/Ctrl + M
%reset -f

# Import Library

In [ ]:
from requests import get       # ใช้เรียก API จากเว็บไซต์ (HTTP GET request)
import pandas as pd            # จัดการ DataFrame, แปลง JSON เป็นตาราง


# Configure Coordinates and Files

In [ ]:
# --------- ตั้งค่าพิกัดและไฟล์ ---------
lat, lon = 13.7563, 100.5018             # พิกัดกรุงเทพฯ
file_name = "air_quality.xlsx"           # ชื่อไฟล์ Excel สำหรับเก็บข้อมูลสะสม


# Retrieve Data for the Past 90 Days

In [ ]:
# --------- ดึงย้อนหลัง 90 วัน ---------
# สร้าง URL API พร้อมพารามิเตอร์
url = (
    "https://air-quality-api.open-meteo.com/v1/air-quality"
    f"?latitude={lat}&longitude={lon}"
    "&hourly=pm10,pm2_5"
    "&past_days=90"
    "&timezone=Asia/Bangkok"          # ให้ API ส่งเวลาเป็น local Bangkok
)

# เรียก API และแปลง JSON เป็น DataFrame
api = get(url)
api_json = api.json()
df = pd.DataFrame(api_json["hourly"])

# แปลง df["time"] เป็น tz-aware Bangkok (เพื่อเปรียบเทียบเวลาได้ถูกต้อง)
df["time"] = pd.to_datetime(df["time"]).dt.tz_localize("Asia/Bangkok")

# เวลาปัจจุบันปัดชั่วโมงล่าสุด → tz-aware Bangkok
cut_time = pd.Timestamp.now(tz="Asia/Bangkok").floor("h") - pd.Timedelta(hours=1)

# ตัด forecast ออก → เหลือเฉพาะเวลาที่ยืนยันแล้ว
df = df[df["time"] <= cut_time]

# ลบ timezone ก่อน export Excel เพราะ Excel ไม่รองรับ tz-aware
df["time"] = df["time"].dt.tz_localize(None)

# เขียนข้อมูลลง Excel
df.to_excel(file_name, index=False)
print(f"สร้างไฟล์ใหม่พร้อมข้อมูลย้อนหลัง 90 วันจนถึง {cut_time} ✅ (เวลาย้อนหลัง 1 ชม.เทียบกับเวลา ณ ปัจจุบัน เพื่อการยืนยันข้อมูลที่ถูกต้อง)")

# อ่าน/แสดง ข้อมูลใน Excel
df = pd.read_excel(file_name)
print(df)


สร้างไฟล์ใหม่พร้อมข้อมูลย้อนหลัง 90 วันจนถึง 2025-09-10 02:00:00+07:00 ✅ (เวลาย้อนหลัง 1 ชม.เทียบกับเวลา ณ ปัจจุบัน เพื่อการยืนยันข้อมูลที่ถูกต้อง)
                    time  pm10  pm2_5
0    2025-06-12 00:00:00  19.7   16.5
1    2025-06-12 01:00:00  18.0   15.3
2    2025-06-12 02:00:00  15.4   13.5
3    2025-06-12 03:00:00  12.9   11.5
4    2025-06-12 04:00:00  10.3    9.5
...                  ...   ...    ...
2158 2025-09-09 22:00:00   5.2    5.1
2159 2025-09-09 23:00:00   5.2    5.1
2160 2025-09-10 00:00:00   4.4    4.2
2161 2025-09-10 01:00:00   3.7    3.5
2162 2025-09-10 02:00:00   3.5    3.2

[2163 rows x 3 columns]
